## 1 Speichern des trainierten ML-Models

### 1.1 Fürs Speicher des Models:

    saver = tf.train.Saver()
    saver.save(session, path_to_your_model)
Per Default werden alle Variable von Graph in der Datei gespeichert. Falls du nicht alle Variables brauchst, kannst auch die zu speichernden Variable über tf.train.Saver() als Python Dictionary übergeben z.B. 

    v1 = tf.get_variable("v1", [3], initializer = tf.zeros_initializer)
    v2 = tf.get_variable("v2", [5], initializer = tf.zeros_initializer)
    saver = tf.train.Saver({"v2": v2})

Hier wird nur v2 gespeichert.
### 1.2 Model Speichern nach bestimmter Iterationen 
Es wird oft in Deep Learning erst nach bestimmten Iterationen (z.B. 1000 steps) das Model zu speichern. Durch Parameter **global_step** wird die Anzahl der Iteration bestimmt.

    saver.save(sess, './modelname',global_step=1000)
Dieses Model wird durch suffix "-100" gekennzeichnet

### 1.3 Model Speichern periodisch
In der Realität wird Model auch oft periodisch gepeichert z.B. alle zwei Stunden und zwar speichern nur die letzten fünf Model-Datei:
    
    tf.train.Saver(max_to_keep=5, keep_checkpoint_every_n_hours=2)
Tensorflow speichert per default nur die letzten fünf Model. Wenn man mehr Modelle speichern möchte, kann durch Parameter **max_to_keep** bestimmen



## 2 Laden vom trainierten Model:
### 2.1 Laden via import_meta_graph
wie besprochen Graph besagt, aus welchen Komponenten(Tensor, Op, Variable und Konstant) ein Model gebaut wird. Bei der Ausführung wird pro Session mindestens eine Graph benötigt. Aber wir müssen nicht jedes mal diese Graph noch mal implementieren... Wenn wir das trainierte Model schon mal gespeichert haben, liegt schon die Graph des Models in .meta Datei.
Wir können die Graph durch **import_meta_graph** wieder herstellen

Um die Rechnung der Daten von Graph zu gestalten, wird hier auch eine **Session** benötigt. In der Session können wir vordefinierte Variables (k, b) wieder aufrufen:

    with tf.Session() as sess:
        sess.run(init)
        saver = tf.train.import_meta_graph('./model_all/model.ckpt.meta')
        saver.restore(sess, tf.train.latest_checkpoint('./model_all'))
        k, b = sess.run([k,b])
        
### 2.2 Variable Übergabe 
You can also parse a variable by dictionary to perform prediction.

    with tf.Session() as sess:
        sess.run(init)
        saver = tf.train.import_meta_graph('./model_all/model.ckpt.meta')
        saver.restore(sess, tf.train.latest_checkpoint('./model_all'))
        sess.run(predict_weight, feed_dict = {x: 1.90}))

In training part we defined variables for parameter **predict_weight**

    x = tf.Variable(0.)                   # define a value to parse
    mult_var = tf.multiply(k, x)          # k * x
    predict_weight = tf.add(mult_var, b)  # k * x + b


## 3 Model Dateien
Für jedes Model wird per default vier neuen Dateien erzeugt
* checkpoint
* modelname.ckpt.data-00000-of-00001
* modelname.ckpt.index
* modelname.ckpt.meta

### 3.1 checkpoint
checkpoit Datei führt allen Dateien vom gespeicherten Model als List zusammen und wird vom tf.train.Saver() gepflegt.
### 3.2 model.ckpt.meta
In Datei "model.ckpt.meta" wird Meta-Data von Graph  gespeichert z.B. Meta-Daten von Operationen und Tensor.
### 3.3 model.ckpt
Hier wird die Daten von Variables als SSTable-Format gespeichert (SSTable is ähnlich wie key, value pair)

for more: 
https://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/